In [12]:
import pandas as pd
import glob,os
import re
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append(os.path.expanduser("~") + "/collproc/")
from raw import *
from coll import *

In [2]:
df = pd.read_csv("genome_roughness_768_250_501_21_divide.csv")

In [3]:
df['fbase'] = [os.path.basename(item)[:-4] for item in df.localpath]

In [4]:
df['samploc'] = [item.split("/")[-2] for item in df.localpath]

In [5]:
df['catalog'] = df.fbase.apply(lambda x: parse_sample_id(x)['catalog'])
df['manbran'] = df.fbase.apply(lambda x: parse_sample_id(x)['manbran'])
df['mtrial'] = df.fbase.apply(lambda x: parse_sample_id(x)['mtrial'])

#### Correct manbrans that were skipped because only one letter

In [6]:
df.manbran[df.manbran.isnull()] = df.fbase[df.manbran.isnull()].apply(lambda x: x.split("_")[1])

#### Correct mtrials

In [7]:
df.mtrial[df.fbase=='2172_kb_'] = '2'

In [8]:
df.mtrial[df.fbase=='97_kv_166'] = '0'
df.mtrial[df.fbase=='97_kv_167'] = '1'

In [9]:
df.mtrial[df.fbase=='2982_kp_'] = '2'

# 2023-24 Campaign

In [15]:
sys.path.append("/Users/damoncrockett/ivpy/src")
from ivpy import *
from ivpy.extract import extract

In [14]:
allnew = glob.glob(os.path.join("/Users/damoncrockett/lmlproc/data/campaign2023-2024/","*.tif"))

In [16]:
nf = pd.DataFrame({"fname":allnew})

In [20]:
nf['fbase'] = [os.path.basename(item)[:-4] for item in nf.fname]

In [21]:
nf['catalog'] = nf.fbase.apply(lambda x: parse_sample_id(x)['catalog'])
nf['manbran'] = nf.fbase.apply(lambda x: parse_sample_id(x)['manbran'])
nf['mtrial'] = nf.fbase.apply(lambda x: parse_sample_id(x)['mtrial'])

In [25]:
attach(nf,'fname')

In [27]:
nf['roughness'] = extract("roughness")

5% 10% 15% 20% 25% 30% 35% 40% Image too small to process: /Users/damoncrockett/lmlproc/data/campaign2023-2024/5615a_lb_0.tif
unsupported operand type(s) for /: 'NoneType' and 'int'
45% 50% 55% 60% 65% 70% Image too small to process: /Users/damoncrockett/lmlproc/data/campaign2023-2024/5615b_lb_1.tif
unsupported operand type(s) for /: 'NoneType' and 'int'
75% 80% 85% 90% 95% 

In [41]:
nf = nf.loc[~nf.roughness.apply(lambda x: isinstance(x,list))]

In [44]:
nf.roughness = nf.roughness.apply(float)

# Join

In [47]:
nf.rename(columns={"fname":"localpath"},inplace=True)

In [69]:
nf['samploc'] = 'camp24'

In [74]:
nf.mtrial = nf.mtrial.fillna('0')

In [75]:
#show(nf.localpath.loc[nf.roughness < 0.01])

In [76]:
af = pd.concat([df,nf])

In [77]:
af.to_pickle('raw.pkl')